# Data Science for the Automotive Industry: Third practical session - RL

Reinforcement learning (RL) is an area of machine learning concerned with how intelligent agents ought to take actions in an environment in order to maximize the notion of cumulative reward (Wikipedia).

In this session, we will grasp a feeling of a set of different reinforcement learning techniques:

1) Thomson sampling

2) Tabular methods

3) Reinforcement learning based on NN

<!-- References:

https://towardsdatascience.com/reinforcement-learning-w-keras-openai-dqns-1eed3a5338c

https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_12_01_ai_gym.ipynb

https://medium.com/analytics-vidhya/rendering-openai-gym-environments-in-google-colab-9df4e7d6f99f 
-->

Developed by Nicolas Gutierrez in February 2022

## Installing and importing all required packages

Installing some extra packages we will need throughout the class.

In [ ]:
### Do not modify this cell, not an exercise

# Libraries for plotting and loading environments
!pip3 install box2d-py
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install colabgymrender==1.0.2

Importing required libraries

In [ ]:
### Do not modify this cell, not an exercise

# Standard libraries
import os
import glob
from collections import deque
import time
from datetime import datetime
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import beta
import pylab as pl
from IPython import display
import gym

# NNs
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

# Utilities
from colabgymrender.recorder import Recorder

Giving colab access to our google drive, so we can load the datasets and write results. A windows will pop up asking for permissions.

In [ ]:
### Exercise 1: Mount google drive

# You will need two lines of code below


#

## Thomson Sampling

Imagine you are working for Google, specifically for the advertising/marketing department. You have to give some advice to one of your customers that is a car seller. 

You are required to tell them what is the car that people like the most. This is not a survey and you are worried that decisions may be biased if you do that. You can use google platform to show announces one at a time and measure the success of the model by mesuring how often people clicks on it.

![](https://drive.google.com/uc?export=view&id=1vEIBRzZWEs71ROACfsMi5z9mVS48sIK7)

You have 10000 tries as a maximum but you will be rewarded if you use the minimum amount of them. 

What would you do?

### Dataset loading

In [ ]:
### Exercise 2: Locate the file dataset.csv in your google drive

# Use the required library and method and find the path
list_of_files = 
#

print(list_of_files)

## If you did it correctly you will see a list with one file where the basename
## is dataset.csv

We have 10 ads that can be displayed to a user. Each time a user connects to the website, that makes a round or increases the index by one. At each index or round we can select just one ad to show to the user.


If the result is 1 is that the user liked the ad and clicked it. Otherwise it is a 0.

In [ ]:
### Exercise 3: Load the file with pandas and print the first 5 files

# Load the file in the folowing line
ads_clicking = 
#

# Show the 5 first lines in the following

#

### Dataset exploration
Let's have a look at the dataset. Even if in this situation this is not realistic, because in the hypothetic case described, we will never have accessed to all this data, we will plot the dataset in order to detect which one is the most successful ad.

In [ ]:
### Exercise 4: Show a bar plot with the number of clicks per ad

# Display a bar plot where the horizontal axis will have the Ads and the vertical
# axis will have the number of clicks

#

# Include title and names of the axis



#

## Which one is the most successful ad?

### Model creation
In this section, we will create a small model that will allow us detecting which is the most successful ad on the fly without having access to the whole data, just randomly sampling between all available options.

In [ ]:
### Exercise 5: Initialise accumulation variables

# Initialise the following variables being:
# N -> Number of rows in ads_clicking
# d -> Number of columns in ads_clicking
# numbers_of_rewards_1 -> a list of as many zeros as d
# numbers of rewards_0 -> a list of as many zeros as d
# total_reward -> an integer 0

# Complete the following lines 
N, d = 
ads_selected = 
numbers_of_rewards_1 = 
numbers_of_rewards_0 = 
total_reward = 
#

In [ ]:
### Exercise 6: Code the model

for n in range(N):
  # Initilization of variables per round
  ad = 0
  max_random = 0
  
  # Play a round to select one ad
  for i in range(d):
    # Investigate the betavariate distribution and calculate it based on initialised variables
    random_beta = 
    #

    if random_beta > max_random:
      max_random = random_beta
      ad = i
  
  # Evaluate the ad we selected 
  ads_selected.append(ad)
  reward = ads_clicking.values[n, ad]
  if reward == 1:
    numbers_of_rewards_1[ad] += 1
  else:
    numbers_of_rewards_0[ad] += 1
  total_reward += reward

print(f"Total reward is {total_reward}")
## If you did it right total reward should around 2600 (The final result has some randomness!!)
## Careful if you run this cell twice in a row without resetting first the variables
## in the previous cell!! 

Do you understand the logic behind? 

Now, let's plot how our statistical agent is getting an understanding on what ad is the most succesful...

In [ ]:
### Exercise 7: Create a bar plot with the selection of ads from the model

# Complete the following lines (Include, title and names of x and y axis)




#

### Understanding the model

In [ ]:
### Do not modify this cell, not an exercise

def data_generation(numbers_of_rewards_1, numbers_of_rewards_0, n):
  x = []
  y = []
  if n % 100 == 0:
    for i in range(len(numbers_of_rewards_1)):
      a = numbers_of_rewards_1
      b = numbers_of_rewards_0
      x.append(np.linspace(beta.ppf(0.01, a, b), beta.ppf(0.99, a, b), 100))
      y.append(beta.pdf(x[-1], a, b))
  return x, y

def plot_distributions(x, y, n):
  if n % 100 == 0:
    plt.cla()
    for i in range(len(x)):
      pl.plot(x[i], y[i])
    
    pl.title(f"Round {n}")
    pl.legend([f'Ad {j+1}' for j in range(len(x))])
    display.display(pl.gcf())
    display.clear_output(wait=True)
    
    time.sleep(0.05)

In [ ]:
### Exercise 8: Use the previous two methods to extrac x and y and plot them

# Complete the following two lines
x, y = 

#

Finally, let's go into the statistical details of the implementation:

The Beta distribution is best for representing a probabilistic distribution of probabilities: the case where we don't know what a probability is in advance, but we may have some reasonable guesses. Check this link for further details: [Intuition behind beta distribution](https://stats.stackexchange.com/questions/47771/what-is-the-intuition-behind-beta-distribution)

The beta distribution essentially __defines the probability that a Bernoulli experiment's success probability is p given the outcome of the experiment__.

In the previous case, we start with α and β equal 1, where the beta distribution is a uniform distribution. In the first experiments it will just randomly sample the ads, but as soon as it starts hitting a success ad it will modify the distribution shape. According to the beta distribution, the chances of success of the Ad5 are around 0.27.


## Tabular method

In this exercise you will learn techniques based on Monte Carlo estimators to solve reinforcement learning problems in which you don't know the environmental behavior. We have to learn based on an episode by episode strategy and estimate the state-action values over many episodes to find an optimal/good policy. 

As an examble for this we consider the frozenlake environment provided by the gym API. The fozenlake environment is represented by a 4x4 grid consisting of a start grid , some hole grids and one goal grid. 

The agent can move, __up, down, right and left__. As further difficulty the grid is also slippery, meaning that an action might not lead to the desired direction. 

The rewards are set as follows: 
- 0 for each transition not entering the goal grid.
- +1 for reaching the goal grid.

<!-- Reference:
https://www.deep-teaching.org/notebooks/reinforcement-learning/exercise-monte-carlo-frozenlake-gym
-->


### Environment creation and familiarization

In [ ]:
### Do not modify this cell, not an exercise

# Instantiation of the environment
env = gym.make('FrozenLake-v0', is_slippery=True)

# Printing the Action Space and the Observation space
print("Action space: ", env.action_space)
print("Observation space: ", env.observation_space)

In [ ]:
### Do not modify this cell, not an exercise

env.reset() #reset the environment the set agent to start state
env.render() #display the agents current state on the grid

In [ ]:
### Exercise 9: Execute the environment with random actions

env.reset()
for i in range(10):
    # Investigate env class and action_space method to sample a random action
    random_action =  
    #
    print("Action:",random_action)
    
    # Investigate the env class to take the action
    new_state, reward, done, info = 
    #
    print("State:", new_state)
    
    # new_state: new state after action (random_action) taken in current state 
    # reward: reward obtained after taken action (random_action) in current state and entering new state (new_state)
    # done: bool flag, true if goal or hole is reached
    # info: slippery probability, baseline is 1/3
    
    print("-----")
    env.render() # display current agent state
    print(f"Reward value: {reward}\n")
    if done:
        break

The abbreviations stand for

S: Start
F: Frozen (safe)
H: Hole
G: Goal.
The actions are numerated as left (0), right (1), down (2), up (3).

A generic random walk until a terminal state is reached (done == True) is implemented below.

Such a walk, until a terminal state is reached, represents one episode.

### Model creation

In [ ]:
### Exercise 10: Initialise the variables for the model

# Select a suitable num of episodes and epsilon
# TIP: Start with episodes in the order of 10k and check what happens
# epsilon will control the amount of exploration vs explotation, start
# with small values <= 0.15
num_episodes = 
epsilon = 
#

# Initialise Q and n_s_a using numpy.
# Q will hold as many q values as observation_space by action_space size
# n_s_a will hold the times an action was taken based on a observation
# Rest of variables will be empty 
Q = 
n_s_a = 
rList = 
success_rate = 
episode_length = 
#

In [ ]:
### Exercise 11: Build the model

fig, ax = pl.subplots(ncols=2, figsize=np.array([2*6.4, 4.8]))

for i in range(num_episodes):
    # Initialise variables
    # state should reset the environment
    # The rest of variables should be either False/emtpy or 0
    state = 
    done = 
    results_list = 
    result_sum = 
    steps = 
    #

    while not done:
        # Look for a random number generator in numpy
        random_number = 
        #

        # In order to implement exploration vs explotation, code an if
        # else clause that will sample an action randomly if random_number
        # smaller than epsilon and will decide based on Q values otherwise.
        # TIP: look for argmax function and keep in mind you are in a specific 
        # state currently
        if random_number < epsilon:
            action = 
        else:
            action = 
        #
        steps += 1
        new_state, reward, done, _ = env.step(action)
        results_list.append((state, action))
        result_sum += reward
        state = new_state
    
    rList.append(result_sum)
    if i>0 and i<100:
      success_rate.append(sum(rList)/i)
    elif i>=100:
      success_rate.append(sum(rList[-100:])/100)
    if i>0:
      episode_length.append(steps)


    # Complete the heart of the algorithm, the n_s_a and Q values.
    for (state, action) in results_list:
        # Increase the n_s_a indicated by 1.0
        # alpha must be 1.0 divided by the previous n_s_a
        # Q tries to reflect how successful was a movement in the context of
        # achieving the objective of the game
        n_s_a[state, action] += 
        alpha = 
        #
        Q[state, action] += alpha * (result_sum - Q[state, action])

    if i % 1000 == 0 and i is not 0:
          plt.cla()
          ax[0].plot(np.arange(0, i, 100), success_rate[::100], label='Success rate', color=u'#1f77b4', zorder=1)
          ax[0].set_xlabel('Number of episodes')
          ax[0].set_ylabel('Success rate')
          ax[0].set_title("Success rate vs number of episodes")
         
          ax[1].plot(np.arange(0, i, 100), episode_length[::100], label='Episode length', color='y', zorder=1)
          ax[1].set_xlabel('Number of episodes')
          ax[1].set_ylabel('Episode length')
          ax[1].set_title("Episode length vs episodes")

          display.display(pl.gcf())
          display.clear_output(wait=True)
          time.sleep(0.05)

print("Success rate: " + str(sum(rList)/num_episodes))

env.close()

### Model testing

In [ ]:
### Exercise 12: Testing the trained model

state = env.reset()
for i in range(1000):
    
    # Use the same selection method than when training the model, but in this
    # case you don't need exploration
    action = 
    #

    print("Action:", action)
    new_state, reward, done, info = env.step(action) #agent takes action (random_action)
    state = new_state
    
    # new_state: new state after action (random_action) taken in current state 
    # reward: reward obtained after taken action (random_action) in current state and entering new state (new_state)
    # done: bool flag, true if goal or hole is reached
    # info: slippery probability, baseline is 1/3
    
    print("-----")
    env.render() # display current agent state
    print(f"Reward value: {reward}\n")
    if done:
        break

# You will see the model cannot get to the objective every time. Do you know why?


## NN method

### Environment Creation and familiarization

In [ ]:
### Exercise 13: Select a folder for saving your video in google drive

# Modify the following line
video_path = '/content/drive/MyDrive/...'
#

In [ ]:
### Do not modify this cell, not an exercise

env = gym.make("MountainCar-v0")
env = Recorder(env, video_path)

observation = env.reset()
terminal = False
while not terminal:
  action = env.action_space.sample()
  observation, reward, terminal, info = env.step(action)

env.play()

### Configuring/checking Tensorflow

In [ ]:
### Do not modify this cell, not an exercise

tf.test.gpu_device_name()

In [ ]:
### Do not modify this cell, not an exercise

tf.compat.v1.disable_eager_execution()

### Model Creation

Finally, in this exercise we will try to develop a Model that is able to tackle more complicated problems. 

Previously we used MonteCarlo approach in tabular methods. Being a "tabular" space, there is a finite amount of states where our system can be, so we can actually visit them all and check how good the agent does.

In this case, the state space is infinite/continous, so MonteCarlo approaches are not a good way of solving it. We will use a NN to "map" the state-reward space.

Additionally, you will see how a class is coded in Python and how this is normally developed in a conventional research workflow.

In [ ]:
### Exercise 14: Model class programming

class DQN:
    def __init__(self, env, mem_len, gamma, epsilon, epsilon_decay, lr, tau, batch_size):
        self.env     = env
        
        self.gamma = gamma
        self.epsilon = epsilon # Fraction of exploration
        self.epsilon_min = 0.01
        self.epsilon_decay = epsilon_decay
        self.learning_rate = lr
        self.tau = tau

        self.batch_size = batch_size

        self.model        = self.create_model()
        self.target_model = self.create_model()
        
        self.__initialize_memories(mem_len)
        
    # Do you know why the double underscore in the beginning?
    def __initialize_memories(self, mem_len):
        # Memories in this case are a buffer of the latest action-results. We 
        # need to maintain a certain size of the buffer to avoid overflow or
        # too slow memory operations. This could be done with a simple list
        # and some logic to remove new registries if required, but can you find
        # a more suited variable type?
        # TIP: check Python collections library!
        
        # Initialise the following class variables
        self.memory_state = 
        self.memory_action = 
        self.memory_reward = 
        self.memory_new_state = 
        self.memory_done = 
        # 

    def create_model(self):
        # Shape of the inputs of the model
        state_shape  = self.env.observation_space.shape
        
        # Create a sequential tensorflow model with one dense layer, which
        # input is state_shape[0] and activation is relu. The output layer
        # is dense, with as many neurons as the action_space size and 
        # linear activation
        model = 
        

        #

        # compile the model with loss MSE and adam optimizer, keep in mind
        # the learning rate defined in the init function
        

        #

        return model

    def act(self, state):
        # Exploration vs exploitation
        self.epsilon *= self.epsilon_decay
        self.epsilon = max(self.epsilon_min, self.epsilon)
        
        # Use the same logic for decision making as in the tabular methods, but 
        # in this case, instead of a Q table you will need to select from
        # the prediction of the model
        random_number = 
        if random_number < self.epsilon:
            output = 
        else:
            output = 
        #

        return output

    def remember(self, state, action, reward, new_state, done):
        # Storing every variable in separate deques
        self.memory_state.append(np.array(state)[0])
        self.memory_action.append(action)
        self.memory_reward.append(reward)
        self.memory_new_state.append(np.array(new_state)[0])
        self.memory_done.append(done)

    def replay(self):
        if len(self.memory_state) > self.batch_size: 
            # Get indices of samples for replay buffers
            indices = np.random.choice(range(len(self.memory_state)), 
                                       size=self.batch_size)
            
            # Generate the subsampling of all variables
            state = np.array(self.memory_state)[indices]
            new_state = np.array(self.memory_new_state)[indices]
            action = np.array(self.memory_action)[indices]
            reward = np.array(self.memory_reward)[indices]
            done = np.array(self.memory_done)[indices]
            
            # Use the target_model to predict state and new_state
            target = self.target_model.predict(state)
            q_inter = self.target_model.predict(new_state)
            #

            for i in range(len(target)):
                if done[i]:
                    target[i][action[i]] = reward[i]
                else:
                    q_future = max(q_inter[i])
                    target[i][action[i]] = reward[i] + q_future * self.gamma
            
            self.model.fit(state, target, epochs=1, verbose=0)

    def target_train(self):
        weights = self.model.get_weights()
        target_weights = self.target_model.get_weights()
        for i in range(len(target_weights)):
            target_weights[i] = weights[i] * self.tau + target_weights[i] * (1 - self.tau)
        self.target_model.set_weights(target_weights)

    def save_model(self, fn):
        self.model.save(fn)

def main():
    env     = gym.make("MountainCar-v0")
    gamma   = 0.85
    epsilon = 1
    epsilon_decay = 0.999
    lr = 0.005
    tau = 0.125
    
    mem_len = 2048
    
    episodes  = 500
    trial_len = 500

    batch_size = 64

    dqn_agent = DQN(env, mem_len, gamma, epsilon, epsilon_decay, lr, tau, batch_size)
    
    for episode in range(episodes):
        tic = time.time()
        print(f"Episode: {episode}/{episodes}")
        
        episode_reward = 0
        cur_state = env.reset().reshape(1,2)
        for step in range(trial_len):
            # Model action
            action = dqn_agent.act(cur_state)
            
            # Environment actioned
            new_state, reward, done, _ = env.step(action)

            # reward = reward if not done else -20
            new_state = new_state.reshape(1,2)
            dqn_agent.remember(cur_state, action, reward, new_state, done)
            
            dqn_agent.replay()       # internally iterates default (prediction) model
            if step % 4 == 0:
                dqn_agent.target_train() # iterates target model

            cur_state = new_state

            episode_reward += reward
            if done or reward != -1:
                break
        print(f"\t Current reward: {episode_reward}")
        print(f"\t Episode took {time.time()-tic:.2f}s")
        if step >= 199:
            print(f"\t Failed to complete in episode {episode}")
        else:
            now = datetime.now() # current date and time
            date_time = now.strftime("%Y%m%d_%H%M%S")
            print(f"\t ------------- Completed in episode {episode}, step {step} ------------- ")
            
            # Find a suitable folder for saving your model
            dqn_agent.save_model(f"/content/drive/MyDrive/.../{date_time}_success_{episode}-{step}.model")
            #

if __name__ == "__main__":
    main()

### Model Testing

In [ ]:
### Exercise 15: Load the most promising model

model = tf.keras.models.load_model(f"/content/drive/MyDrive/...")

In [ ]:
### Do not modify this cell, not an exercise 

env = gym.make("MountainCar-v0")
env = Recorder(env, video_path)

observation = env.reset()
done = False
while not done:
  prediction = model.predict(observation.reshape(1, 2))
  action = np.argmax(prediction[0])
  observation, reward, done, info = env.step(action)

env.play()